In [5]:
import nltk
nltk.download("punkt")

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

import tensorflow as tf
import numpy as np
import random,json

AttributeError: partially initialized module 'nltk' has no attribute 'data' (most likely due to a circular import)

In [ ]:
# Using google colab's files method to upload json file which contains the data 

# from google.colab import files
# files.upload()


In [ ]:
with open("intents.json",'r') as data:
  intents = json.load(data)

In [ ]:
intents

In [ ]:
words = []
classes = []
documents = []
ignore = ["?"]

In [ ]:
# data preprocessing: looping through json file which is saved in variable 
# intents and tokenizing the patterns, extract word with it's tag and saving 
# them in the list.

for intent in intents["intents"]:
  for pattern in intent["patterns"]:
    word = nltk.word_tokenize(pattern)
    words.extend(word)

    documents.append((word,intent["tag"]))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])



In [ ]:
words = [stemmer.stem(word.lower()) for word in words if word not in ignore]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))


In [ ]:
print("Total documents:", len(documents))
print("Total classes:", len(classes))
print("Total unique words:", len(words))

In [ ]:
# creating lists for training the data

training = [] # f(x)
output = [] # f(y)

output_empty = [0] * len(classes)

In [ ]:
#test

documents[0]

In [ ]:
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy']) 



In [ ]:
model.fit(np.array(train_x), np.array(train_y), epochs=1000, batch_size=8, verbose=1)
model.save("model.pkl")

In [ ]:
# prediction

import pickle
pickle.dump({"words":words,"classes":classes},open("training_data","wb"))



In [ ]:
from keras.models import load_model
model = load_model("model.pkl")

In [ ]:
with open("training_data","rb") as data:
  data = pickle.load(data)
  words = data["words"]
  classes = data["classes"]


In [ ]:
with open("intents.json",'r') as data:
  intents = json.load(data)

In [ ]:
def cleanUp(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

In [ ]:
def bow(sentence, words):
  sentence_words = cleanUp(sentence)
  bag = [0] * len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w==s:
        bag[i] = 1
  bag = np.array(bag)
  return (bag)

In [ ]:
ERR_THRESHOLD = 0.30

def classify(sentence):
  bag = bow(sentence,words) # generating all the possibilties from a sentence
  result = model.predict(np.array([bag]))

  result = [[i,r] for i,r in enumerate(result[0])]
  result.sort(key=lambda x: x[1], reverse=True)

  result_li = []
  for res in result:
    if res[1] > ERR_THRESHOLD:
      result_li.append((classes[res[0]],res[1])) #intent and probability

  return result_li




In [ ]:
def get_response(sentence):
  res = classify(sentence)
  if res:
    while res:
      for intent in intents["intents"]:
        if intent["tag"] == res[0][0]:
          return print(random.choice(intent["responses"])) 

      res.pop(0)


In [ ]:
get_response("Who are you?")
